In [76]:
import numpy as np
import pandas as pd
from scipy.stats import circmean, circstd
from statsmodels.stats.diagnostic import kstest_normal
import matplotlib.pyplot as plt

# Code for various statistical analysis

In [77]:
# Define functions for circular statistics calculations
def compute_stability_measure(itis):
    phases = len(itis)
    iti_angles = np.deg2rad(np.cumsum(itis) * (360 / 60))
    return circstd(iti_angles) / circmean(iti_angles) * np.sqrt(phases)

def compute_synchronisation_measure(itis):
    iti_angles = np.deg2rad(np.cumsum(itis) * (360 / 60))
    return circstd(iti_angles) / circmean(iti_angles)

def compute_tempo_change_measure(itis):
    print(itis)
    return (np.mean(itis[-10:]) - np.mean(itis[:10])) / np.mean(itis[:10])

# Load tapping data from a file with moving average
def load_tapping_data(file_path):
    tapping_times = None
    with open(file_path, "r") as file:
        tapping_times = file.read()
    time_points = []
    for line in tapping_times.split('\n'):
        if line.strip():
            time_points.append(float(line.split()[0]))
    tapping_intervals = np.diff(time_points)

    # Compute moving average with window size 5
    # moving_average = np.convolve(tapping_intervals, np.ones(5)/5, mode='valid')
    moving_average = tapping_intervals
    return moving_average

# List of Participants

In [78]:
# Define file paths and tempo information
all_participants = ['Arjun', 'Mahika', 'Mihika', 'Vanshita', 'Shiven', 'Poorvi', 'Sriteja']
trained_participants = ['Shiven', 'Poorvi', 'Sriteja']
untrained_participants = ['Arjun', 'Mahika', 'Mihika', 'Vanshita']
song_ids = ['A', 'B', 'C']
song_tempos = {'A': 122, 'B': 51, 'C': 109}

In [79]:
# Create a DataFrame to store participant rankings
participant_data = []

# Perform analysis for all participants
for participant in all_participants:
    file_paths = [f'/home/ujjwal-shake-her/mmt/MMT-project/timestamp_files/timestamps_{participant}_{song_id}.txt' for song_id in song_ids]
    participant_entry = {'Participant': participant}

    for song_id, file_path in zip(song_ids, file_paths):
        tapping_intervals = load_tapping_data(file_path)

        if len(tapping_intervals) < 1:
            print(f"No valid tapping data found for Song {song_id} of Participant {participant}. Skipping analysis.")
            continue

        # Compute stability measure
        stability_measure = compute_stability_measure(tapping_intervals)
        participant_entry[f'Stability ({song_id})'] = stability_measure

        # Compute synchronisation measure
        synchronisation_measure = compute_synchronisation_measure(tapping_intervals)
        participant_entry[f'Synchronisation ({song_id})'] = synchronisation_measure

        # Compute tempo change measure
        tempo_change_measure = compute_tempo_change_measure(tapping_intervals)
        participant_entry[f'Tempo Change ({song_id})'] = tempo_change_measure

        # Compute variance from real BPM
        tempo_variance = np.abs(song_tempos[song_id] - (60 / np.mean(tapping_intervals)))
        participant_entry[f'Tempo Variance ({song_id})'] = tempo_variance

    participant_data.append(participant_entry)

# Create DataFrame from participant data
participant_rankings = pd.DataFrame(participant_data)

# Rank participants based on their average measures
participant_rankings['Average Stability'] = participant_rankings.filter(like='Stability').mean(axis=1)
participant_rankings['Average Synchronisation'] = participant_rankings.filter(like='Synchronisation').mean(axis=1)
participant_rankings['Average Tempo Change'] = participant_rankings.filter(like='Tempo Change').mean(axis=1)
participant_rankings['Average Tempo Variance'] = participant_rankings.filter(like='Tempo Variance').mean(axis=1)

participant_rankings['Rank Stability'] = participant_rankings['Average Stability'].rank(ascending=False)
participant_rankings['Rank Synchronisation'] = participant_rankings['Average Synchronisation'].rank(ascending=False)
participant_rankings['Rank Tempo Change'] = participant_rankings['Average Tempo Change'].rank(ascending=True)
participant_rankings['Rank Tempo Variance'] = participant_rankings.filter(like='Average Tempo Variance').rank(ascending=True)

# Display the rankings in a table format
ordered_participant_rankings = participant_rankings.sort_values(by='Average Tempo Variance', ascending=True)
ordered_participant_rankings.head(10)
# print(participant_rankings[['Participant', 'Average Stability', 'Rank Stability',
#                             'Average Synchronisation', 'Rank Synchronisation',
#                             'Average Tempo Change', 'Rank Tempo Change']])

[0.436 0.471 0.486 0.464 0.5   0.493 0.515 0.464 0.485 0.543 0.457 0.525
 0.447 0.485 0.45  0.508 0.507 0.507 0.449 0.486 0.493 0.524 0.497 0.486
 0.472 0.506 0.464 0.558 0.442 0.529 0.493 0.472 0.485 0.478 0.507 0.472
 0.543 0.457 0.536 0.492 0.493 0.5   0.501 0.493 0.492 0.543 0.457 0.507
 0.486 0.464 0.528 0.529 0.486 0.55  0.485 0.443 0.5   0.479 0.528 0.529
 0.564 0.479 0.508 0.507 0.464 0.5   0.508 0.565 0.478 0.572 0.508 0.543
 0.557 0.521 0.501 0.493 0.556 0.458 0.607 0.535 0.558 0.55  0.564 0.55
 0.507 0.493 0.586 1.492 0.593 0.45  0.514 0.529 0.514 0.465 0.493 0.549
 0.558 0.459 0.476 0.523 0.499 0.464 0.471 0.515 0.485 0.472 0.493 0.557
 0.478 0.486 0.522 0.493 0.514 0.492 0.536 0.543 0.464 0.507 0.515 0.535
 0.508 0.499 0.493 0.479 0.507 0.493 0.492 0.493 0.522 0.486 0.507 0.549
 0.486 0.529 0.457 0.486 0.478 0.458 0.507 0.529 0.535 0.507 0.528 0.508
 0.564 0.543]
[0.857 1.55  0.807 1.529 0.857 0.987 1.072 1.243 1.079 1.208 1.214 1.228
 1.308 1.207 1.143 1.121 1.093 1.135 1

,Participant,Stability (A),Synchronisation (A),Tempo Change (A),Tempo Variance (A),Stability (B),Synchronisation (B),Tempo Change (B),Tempo Variance (B),Stability (C),...,Tempo Change (C),Tempo Variance (C),Average Stability,Average Synchronisation,Average Tempo Change,Average Tempo Variance,Rank Stability,Rank Synchronisation,Rank Tempo Change,Rank Tempo Variance
1,Mahika,29.654160,2.421252,-0.004412,0.583474,23.983605,3.096270,-0.001108,0.217813,29.991616,...,0.023452,0.242099,27.876461,2.702801,0.005977,0.347795,1.0,1.0,3.0,1.0
6,Sriteja,26.194368,2.131667,-0.027234,0.847851,21.315636,2.775059,-0.105307,2.864241,31.903539,...,-0.036894,2.457138,26.471181,2.547478,-0.056478,2.056410,3.0,3.0,1.0,2.0
0,Arjun,25.689948,2.126115,0.061767,4.586318,19.988159,2.580460,0.149075,1.376997,27.715877,...,0.035050,0.348089,24.464661,2.363993,0.081964,2.103801,5.0,4.0,6.0,3.0
4,Shiven,26.398074,2.169908,0.086642,3.278661,19.409944,2.465065,0.059255,0.488092,24.602428,...,0.062786,2.724085,23.470149,2.258780,0.069561,2.163613,6.0,6.0,5.0,4.0
3,Vanshita,29.803782,2.386212,0.043487,2.050734,20.908792,2.554416,-0.122655,3.103523,24.046659,...,0.075259,3.759222,24.919744,2.344540,-0.001303,2.971160,4.0,5.0,2.0,5.0
2,Mihika,25.161974,2.104150,0.074059,6.511327,14.824225,1.980970,0.139333,5.224057,27.884983,...,0.048024,0.352595,22.623727,2.158746,0.087139,4.029326,7.0,7.0,7.0,6.0
5,Poorvi,26.878322,2.201958,0.098969,2.992000,19.794071,2.576969,0.034362,3.792447,33.120615,...,0.032929,6.216274,26.597669,2.568801,0.055420,4.333573,2.0,2.0,4.0,7.0


In [80]:
# Perform analysis for trained and untrained participants
def perform_analysis(participants, group_name):
    print(f'Group: {group_name}')
    for participant in participants:
        file_paths = [f'timestamp_files/timestamps_{participant}_{song_id}.txt' for song_id in song_ids]
        print(f'Participant: {participant}')

        for song_id, file_path in zip(song_ids, file_paths):
            tapping_intervals = load_tapping_data(file_path)

            # Compute stability measure
            stability_measure = compute_stability_measure(tapping_intervals)
            print(f'Stability Measure for Song {song_id}: {stability_measure}')

            # Compute synchronisation measure
            synchronisation_measure = compute_synchronisation_measure(tapping_intervals)
            print(f'Synchronisation Measure for Song {song_id}: {synchronisation_measure}')

            # Compute tempo change measure
            tempo_change_measure = compute_tempo_change_measure(tapping_intervals)
            print(f'Tempo Change Measure for Song {song_id}: {tempo_change_measure}')

        print('\n')

perform_analysis(trained_participants, 'Trained Participants')
perform_analysis(untrained_participants, 'Untrained Participants')

Group: Trained Participants
Participant: Shiven
Stability Measure for Song A: 26.398074094207725
Synchronisation Measure for Song A: 2.1699083239497687
[0.45  0.464 0.45  0.501 0.52  0.472 0.536 0.5   0.478 0.465 0.5   0.529
 0.493 0.5   0.557 0.414 0.485 0.483 0.46  0.493 0.493 0.485 0.514 0.529
 0.471 0.507 0.443 0.478 0.529 0.507 0.485 0.493 0.486 0.492 0.465 0.492
 0.515 0.514 0.493 0.492 0.464 0.479 0.5   0.485 0.543 0.5   0.464 0.436
 0.5   0.535 0.479 0.535 0.457 0.55  0.436 0.493 0.514 0.478 0.479 0.478
 0.486 0.529 0.499 0.472 0.449 0.536 0.486 0.457 0.521 0.521 0.465 0.449
 0.507 0.515 0.528 0.478 0.507 0.493 0.479 0.528 0.465 0.485 0.486 0.493
 0.493 0.516 0.476 0.471 0.507 0.479 0.501 0.502 0.497 0.465 0.521 0.515
 0.557 0.529 0.543 0.5   0.514 0.522 0.507 0.522 0.55  0.542 0.551 0.572
 0.471 0.551 0.543 0.514 0.536 0.529 0.514 0.529 0.502 0.551 0.54  0.493
 0.565 0.514 0.551 0.485 0.551 0.493 0.521 0.486 0.543 0.521 0.478 0.515
 0.614 0.586 0.587 0.641 0.457 0.537 0.541 0.